In [8]:
import pandas as pd
import numpy as np
import re #Lavorare con espressioni regolari sulle stringhe

#Percorso contenente i file GTFS
cartella_dati = 'C:/Users/C.Marino/Desktop/dataset'

#Funzione per caricare e pulire i dati
def carica_e_filtra_dati(cartella_dati):
    #Caricamento dei file utili 
    routes = pd.read_csv(f"{cartella_dati}/routes.txt", sep=",")
    trips = pd.read_csv(f"{cartella_dati}/trips.txt", sep=",")
    stop_times = pd.read_csv(f"{cartella_dati}/stop_times.txt", sep=",")

    #Filtro dei dati per agency_id uguale a 'OP1' e route_type = 3 (autobus) 
    routes['agency_id'] = routes['agency_id'].astype(str).str.strip()
    filtered_routes = routes[(routes['agency_id'] == 'OP1') & (routes['route_type'] == 3)]

    #Join tra routes e trips tramite chiave route_id
    final_join = pd.merge(filtered_routes, trips, on="route_id", how="inner")

    #Join con stop_times tramite chiave trip_id
    final_join_with_stop_times = pd.merge(final_join, stop_times, on="trip_id", how="inner")

    return final_join_with_stop_times

#Pulizia del dataset
def pulizia_dati(df):
    #Gestione valori nulli
    if df.isnull().sum().sum() > 0:
        print("\nValori mancanti per colonna:")
        print(df.isnull().sum())
        df = df.dropna(subset=['trip_id', 'stop_id'])  #Rimuove righe con trip_id e stop_id nulli

    #Convertiamo arrival_time in formato numerico corretto (secondi totali)
    df['arrival_time'] = pd.to_datetime(df['arrival_time'], format='%H:%M:%S', errors='coerce')
    df = df.dropna(subset=['arrival_time'])
    df['arrival_time'] = df['arrival_time'].dt.hour * 3600 + df['arrival_time'].dt.minute * 60 + df['arrival_time'].dt.second

    #Rimuovi i viaggi con durata negativa o orari invalidi
    df = df[df['arrival_time'] >= 0]

    #Controllo tipo di trip_id ed eventuale filtro su hash SHA-1
    #if df['trip_id'].dtype == 'object':
     #df = df[~df['trip_id'].str.match(r'^[a-f0-9]{40}$')]

    return df

#Dataset
def esplorazione_dati(df):
    print("\nDimensioni del dataset:", df.shape)
    print("Colonne del dataset:", df.columns)
    print("Tipi di dati per ciascuna colonna:")
    print(df.dtypes)
    print("\nStatistiche descrittive:")
    print(df.describe())

    #Analisi dei valori unici nelle colonne chiave
    print("\nValori unici per 'trip_id':", df['trip_id'].nunique())
    print("Valori unici per 'stop_id':", df['stop_id'].nunique())

#Funzione per eseguire la  join e la pulizia
def esegui_procedura(cartella_dati):
    #Carica e filtra i dati
    df = carica_e_filtra_dati(cartella_dati)

    #Esplora i dati prima della pulizia
    esplorazione_dati(df)

    #Pulizia dei dati
    df_clean = pulizia_dati(df)

    #Salvataggio Dataset pulito
    output_clean_path = f"{cartella_dati}/risultato_join_con_stop_times_clean.txt"
    df_clean.to_csv(output_clean_path, sep=',', index=False, encoding='utf-8')

    print(f"\nDataset pulito salvato in: {output_clean_path}")

#Esegui la procedura
esegui_procedura(cartella_dati)


C:\Users\C.Marino\AppData\Local\Temp\ipykernel_4064\223125839.py:12: DtypeWarning: Columns (0,1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv(f"{cartella_dati}/trips.txt", sep=",")
C:\Users\C.Marino\AppData\Local\Temp\ipykernel_4064\223125839.py:13: DtypeWarning: Columns (0,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv(f"{cartella_dati}/stop_times.txt", sep=",")



Dimensioni del dataset: (4211577, 26)
Colonne del dataset: Index(['route_id', 'agency_id', 'route_short_name', 'route_long_name',
       'route_type', 'route_url', 'route_color', 'route_text_color',
       'service_id', 'trip_id', 'trip_headsign', 'trip_short_name',
       'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible',
       'exceptional', 'arrival_time', 'departure_time', 'stop_id',
       'stop_sequence', 'stop_headsign', 'pickup_type', 'drop_off_type',
       'shape_dist_traveled', 'timepoint'],
      dtype='object')
Tipi di dati per ciascuna colonna:
route_id                  object
agency_id                 object
route_short_name          object
route_long_name           object
route_type                 int64
route_url                 object
route_color               object
route_text_color         float64
service_id                object
trip_id                   object
trip_headsign             object
trip_short_name          float64
direction_id           